In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from matplotlib.pylab import rcParams
plt.style.use('fivethirtyeight') 
from datetime import datetime
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
rcParams['figure.figsize'] = 10, 6
import os
import warnings
warnings.filterwarnings('ignore')
path = "./Data/"
print(os.listdir("./Data/"))

['sample_submission.csv', 'Test.csv', 'Train.csv']


In [4]:
df = pd.read_csv(path+'Train.csv')
test = pd.read_csv(path+'Test.csv')
sub = pd.read_csv(path+'sample_submission.csv')

In [5]:
df.shape

(33750, 15)

In [6]:
test.shape

(14454, 14)

In [7]:
df.date_time = pd.to_datetime(df.date_time)

In [9]:
df.tail().T

,33745,33746,33747,33748,33749
date_time,2017-05-17 21:00:00,2017-05-17 22:00:00,2017-05-17 22:00:00,2017-05-17 23:00:00,2017-05-17 23:00:00
is_holiday,None,None,None,None,None
air_pollution_index,74,81,138,229,140
humidity,85,70,70,66,63
wind_speed,1,1,1,1,1
wind_direction,328,24,24,42,27
visibility_in_miles,7,2,1,6,8
dew_point,7,2,1,6,8
temperature,287.88,286.95,286.95,285.75,285.75
rain_p_h,0,0,0,0,0


In [10]:
test.tail()

,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description
14449,2018-09-30 19:00:00,None,176,21,4,345,4,4,283.45,0.0,0,75,Clouds,broken clouds
14450,2018-09-30 20:00:00,None,214,95,8,280,6,6,282.76,0.0,0,90,Clouds,overcast clouds
14451,2018-09-30 21:00:00,None,173,63,4,238,1,1,282.73,0.0,0,90,Thunderstorm,proximity thunderstorm
14452,2018-09-30 22:00:00,None,21,57,8,268,7,7,282.09,0.0,0,90,Clouds,overcast clouds
14453,2018-09-30 23:00:00,None,116,70,8,328,6,6,282.12,0.0,0,90,Clouds,overcast clouds


In [11]:
for i in df.columns:
    print(i+ str(df[i].unique().shape))

date_time(28589,)
is_holiday(12,)
air_pollution_index(290,)
humidity(88,)
wind_speed(17,)
wind_direction(361,)
visibility_in_miles(9,)
dew_point(9,)
temperature(5611,)
rain_p_h(353,)
snow_p_h(12,)
clouds_all(60,)
weather_type(11,)
weather_description(38,)
traffic_volume(6462,)


In [12]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_process import ArmaProcess
from sklearn.metrics import mean_squared_error

In [13]:
from plotly import tools
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff
import math

In [14]:
def add_cyclic_datepart(df, field_name:str, prefix:str=None, drop:bool=True, time:bool=False, add_linear:bool=False):
    "Helper function that adds trigonometric date/time features to a date in the column `field_name` of `df`."
    make_date(df, field_name)
    field = df[field_name]
    prefix = ifnone(prefix, re.sub('[Dd]ate$', '', field_name))
    series = field.apply(partial(cyclic_dt_features, time=time, add_linear=add_linear))
    columns = [prefix + c for c in cyclic_dt_feat_names(time, add_linear)]
    df_feats = pd.DataFrame([item for item in series], columns=columns, index=series.index)
    for column in columns: df[column] = df_feats[column]
    if drop: df.drop(field_name, axis=1, inplace=True)
    return df

In [15]:
def make_date(df, date_field:str):
    "Make sure `df[field_name]` is of the right date type."
    field_dtype = df[date_field].dtype
    if isinstance(field_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        field_dtype = np.datetime64
    if not np.issubdtype(field_dtype, np.datetime64):
        df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)

In [16]:
from datetime import date, datetime

In [17]:
df_train = df.copy()

In [18]:
def ifnone(a,b):
    "`a` if `a` is not None, otherwise `b`."
    return b if a is None else a

In [19]:
def cyclic_dt_feat_names(time:bool=True, add_linear:bool=False):
    "Return feature names of date/time cycles as produced by `cyclic_dt_features`."
    fs = ['cos','sin']
    attr = [f'{r}_{f}' for r in 'weekday day_month month_year day_year'.split() for f in fs]
    if time: attr += [f'{r}_{f}' for r in 'hour clock min sec'.split() for f in fs]
    if add_linear: attr.append('year_lin')
    return attr

def cyclic_dt_features(d, time:bool=True, add_linear:bool=False):
    "Calculate the cos and sin of date/time cycles."
    tt,fs = d.timetuple(), [np.cos, np.sin]
    day_year,days_month = tt.tm_yday, calendar.monthrange(d.year, d.month)[1]
    days_year = 366 if calendar.isleap(d.year) else 365
    rs = d.weekday()/7, (d.day-1)/days_month, (d.month-1)/12, (day_year-1)/days_year
    feats = [f(r * 2 * np.pi) for r in rs for f in fs]
    if time and isinstance(d, datetime) and type(d) != date:
        rs = tt.tm_hour/24, tt.tm_hour%12/12, tt.tm_min/60, tt.tm_sec/60
        feats += [f(r * 2 * np.pi) for r in rs for f in fs]
    if add_linear:
        if type(d) == date: feats.append(d.year + rs[-1])
        else:
            secs_in_year = (datetime(d.year+1, 1, 1) - datetime(d.year, 1, 1)).total_seconds()
            feats.append(d.year + ((d - datetime(d.year, 1, 1)).total_seconds() / secs_in_year))
    return feats
import re

In [20]:
from functools import partial 
import calendar

In [21]:
add_cyclic_datepart(df, 'date_time',time=True)

,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,...,date_timeday_year_cos,date_timeday_year_sin,date_timehour_cos,date_timehour_sin,date_timeclock_cos,date_timeclock_sin,date_timemin_cos,date_timemin_sin,date_timesec_cos,date_timesec_sin
0,None,121,89,2,329,1,1,288.28,0.0,0.0,...,0.008583,-0.999963,-7.071068e-01,7.071068e-01,-1.836970e-16,-1.000000e+00,1.0,0.0,1.0,0.0
1,None,178,67,3,330,1,1,289.36,0.0,0.0,...,0.008583,-0.999963,-8.660254e-01,5.000000e-01,5.000000e-01,-8.660254e-01,1.0,0.0,1.0,0.0
2,None,113,66,3,329,2,2,289.58,0.0,0.0,...,0.008583,-0.999963,-9.659258e-01,2.588190e-01,8.660254e-01,-5.000000e-01,1.0,0.0,1.0,0.0
3,None,20,66,3,329,5,5,290.13,0.0,0.0,...,0.008583,-0.999963,-1.000000e+00,1.224647e-16,1.000000e+00,0.000000e+00,1.0,0.0,1.0,0.0
4,None,281,65,3,329,7,7,291.14,0.0,0.0,...,0.008583,-0.999963,-9.659258e-01,-2.588190e-01,8.660254e-01,5.000000e-01,1.0,0.0,1.0,0.0
5,None,23,65,3,328,6,6,291.72,0.0,0.0,...,0.008583,-0.999963,-8.660254e-01,-5.000000e-01,5.000000e-01,8.660254e-01,1.0,0.0,1.0,0.0
6,None,184,64,3,328,7,7,293.17,0.0,0.0,...,0.008583,-0.999963,-7.071068e-01,-7.071068e-01,6.123234e-17,1.000000e+00,1.0,0.0,1.0,0.0
7,None,167,64,3,327,7,7,293.86,0.0,0.0,...,0.008583,-0.999963,-5.000000e-01,-8.660254e-01,-5.000000e-01,8.660254e-01,1.0,0.0,1.0,0.0
8,None,119,63,3,327,6,6,294.14,0.0,0.0,...,0.008583,-0.999963,-2.588190e-01,-9.659258e-01,-8.660254e-01,5.000000e-01,1.0,0.0,1.0,0.0
9,None,161,63,3,326,3,3,293.10,0.0,0.0,...,0.008583,-0.999963,-1.836970e-16,-1.000000e+00,-1.000000e+00,1.224647e-16,1.0,0.0,1.0,0.0


In [22]:
df.columns

Index(['is_holiday', 'air_pollution_index', 'humidity', 'wind_speed',
       'wind_direction', 'visibility_in_miles', 'dew_point', 'temperature',
       'rain_p_h', 'snow_p_h', 'clouds_all', 'weather_type',
       'weather_description', 'traffic_volume', 'date_timeweekday_cos',
       'date_timeweekday_sin', 'date_timeday_month_cos',
       'date_timeday_month_sin', 'date_timemonth_year_cos',
       'date_timemonth_year_sin', 'date_timeday_year_cos',
       'date_timeday_year_sin', 'date_timehour_cos', 'date_timehour_sin',
       'date_timeclock_cos', 'date_timeclock_sin', 'date_timemin_cos',
       'date_timemin_sin', 'date_timesec_cos', 'date_timesec_sin'],
      dtype='object')

In [23]:
for i in df.columns:
    print(i+ str(df[i].unique().shape))

is_holiday(12,)
air_pollution_index(290,)
humidity(88,)
wind_speed(17,)
wind_direction(361,)
visibility_in_miles(9,)
dew_point(9,)
temperature(5611,)
rain_p_h(353,)
snow_p_h(12,)
clouds_all(60,)
weather_type(11,)
weather_description(38,)
traffic_volume(6462,)
date_timeweekday_cos(7,)
date_timeweekday_sin(7,)
date_timeday_month_cos(102,)
date_timeday_month_sin(108,)
date_timemonth_year_cos(11,)
date_timemonth_year_sin(11,)
date_timeday_year_cos(666,)
date_timeday_year_sin(687,)
date_timehour_cos(22,)
date_timehour_sin(21,)
date_timeclock_cos(11,)
date_timeclock_sin(11,)
date_timemin_cos(1,)
date_timemin_sin(1,)
date_timesec_cos(1,)
date_timesec_sin(1,)


In [24]:
from fastai.tabular import *

In [25]:
df.wind_direction = df.wind_direction.apply(lambda x: abs(x-360) if x>180 else x)

In [26]:
df.wind_direction

0         31
1         30
2         31
3         31
4         31
5         32
6         32
7         33
8         33
9         34
10        34
11        34
12        35
13        35
14        36
15        36
16        36
17        37
18        37
19        38
20        38
21        38
22        39
23        39
24        40
25        40
26        40
27        10
28        40
29        30
        ... 
33720     45
33721     45
33722     20
33723     20
33724    180
33725     45
33726     19
33727     19
33728     25
33729     42
33730     27
33731     26
33732     26
33733     26
33734     23
33735     23
33736     23
33737     39
33738     39
33739     39
33740     34
33741     34
33742     34
33743     32
33744     32
33745     32
33746     24
33747     24
33748     42
33749     27
Name: wind_direction, Length: 33750, dtype: int64

In [36]:
cat_names = ["date_timemonth_year_cos", "date_timemonth_year_sin",'date_timeweekday_cos','date_timeweekday_sin','date_timeclock_cos', 'date_timeclock_sin','weather_description','is_holiday','wind_speed','weather_type']

In [37]:
cont_names = df.drop(columns=cat_names).columns

In [44]:
df['traffic_volume'] = df['traffic_volume'].astype(float)

In [34]:
dep_var = 'traffic_volume'

In [31]:
procs = [Categorify,FillMissing,Normalize]

In [32]:
test = TabularList.from_df(df.copy(), cat_names=cat_names,cont_names = cont_names)

In [45]:
data = (TabularList.from_df(df,  cat_names=cat_names,cont_names=cont_names,procs=procs)
                           .split_by_rand_pct(valid_pct=0.2)
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [46]:
learn = tabular_learner(data, layers=[1000,500,100], metrics=root_mean_squared_error)

In [50]:
learn.fit(3, 1e-5)

epoch,train_loss,valid_loss,root_mean_squared_error,time
0,14555100.000000,14531834.000000,3807.178467,01:04
1,14573264.000000,14529131.000000,3806.820801,01:02
2,14457728.000000,14527443.000000,3806.597412,01:03


In [53]:
df_train.head().T

,0,1,2,3,4
date_time,2012-10-02 09:00:00,2012-10-02 10:00:00,2012-10-02 11:00:00,2012-10-02 12:00:00,2012-10-02 13:00:00
is_holiday,None,None,None,None,None
air_pollution_index,121,178,113,20,281
humidity,89,67,66,66,65
wind_speed,2,3,3,3,3
wind_direction,329,330,329,329,329
visibility_in_miles,1,1,2,5,7
dew_point,1,1,2,5,7
temperature,288.28,289.36,289.58,290.13,291.14
rain_p_h,0,0,0,0,0
